## Feature Engineering
- time of each trip hr
- length of each trip in km
- 

In [6]:
from haversine import Unit
import haversine as hs
import pandas as pd


In [7]:
import sys
sys.path.append('../scripts/')

from data_cleaning import DataCleaning
from data_extraction import DataExtraction
from feature_extraction import FeatureExtraction
dclean = DataCleaning()
dextract = DataExtraction()
fextract = FeatureExtraction()


In [8]:
clean_df = dextract.load_data('../data/clean_data.csv')
clean_df.head()

,id,order_id,driver_id,driver_action,lat,lng,order_origin,trip_origin,trip_destination,trip_Start_time,trip_end_time
0,1,392001,243828,accepted,6.602207,3.270465,"6.6022066,3.2704649","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
1,2,392001,243588,rejected,6.592097,3.287445,"6.5920972,3.2874447","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
2,3,392001,243830,rejected,6.596133,3.281784,"6.5961334,3.2817841","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
3,4,392001,243539,rejected,6.596142,3.280526,"6.5961416,3.2805263","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
4,5,392001,171653,rejected,6.609232,3.288800,"6.6092317,3.2887999","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36


In [9]:
clean_df = fextract.cordinate_tupple(clean_df,['order_origin','trip_origin','trip_destination'])
clean_df = fextract.convert_to_date(clean_df,['trip_Start_time','trip_end_time'])

In [10]:
features = clean_df.copy()

- calculate distance from the order origin to the driver origin
- calculate trip distance
- calculate trip duration

In [12]:
import requests
import json
# call the OSMR API
r = requests.get(f"http://router.project-osrm.org/route/v1/car/{3.270465},{6.602207};{3.2766339},{6.6010417}?overview=false""")
# then you load the response using the json libray
# by default you get only one alternative so you access 0-th element of the `routes`
routes = json.loads(r.content)
route_1 = routes.get("routes")[0]

route_1

In [13]:

features['driver_order_dst'] = [fextract.distance_from(loc1,loc2) for loc1, loc2 in zip(clean_df.order_origin,clean_df.trip_origin)]
features['trip_dstance'] = [fextract.distance_from(loc1,loc2) for loc1, loc2 in zip(clean_df.trip_origin,clean_df.trip_destination)] 
features['trip_time'] = (clean_df.trip_end_time - clean_df.trip_Start_time).astype('timedelta64[s]')
features['trip_speed'] = features['trip_dstance']/features['trip_time']
features[['driver_order_dst', 'trip_dstance','trip_time','trip_speed']].head()

,driver_order_dst,trip_dstance,trip_time,trip_speed
0,693.62,21048.29,217.0,96.996728
1,1554.09,21048.29,217.0,96.996728
2,788.35,21048.29,217.0,96.996728
3,694.07,21048.29,217.0,96.996728
4,1623.33,21048.29,217.0,96.996728


- calculate year/month/weekdays

In [14]:
# features['year'] = clean_df['trip_Start_time'].dt.year
features = fextract.get_ymwdh(features,'trip_Start_time')
features[['month','day','week_day','hour']]

,month,day,week_day,hour
0,7.0,1.0,3.0,9.0
1,7.0,1.0,3.0,9.0
2,7.0,1.0,3.0,9.0
3,7.0,1.0,3.0,9.0
4,7.0,1.0,3.0,9.0
...,...,...,...,...
1557734,9.0,10.0,4.0,11.0
1557735,9.0,10.0,4.0,11.0
1557736,9.0,10.0,4.0,11.0
1557737,9.0,10.0,4.0,11.0


## get weahter

In [ ]:
# importing the requests library
import requests
  
# api-endpoint
URL = "https://api.weatherbit.io/v2.0/history/daily?"


: 

: 

In [ ]:
weather = {'clouds':[],
           'dewpt':[],
           'wind_spd':[],
           'temp':[],
           'precip':[],
           'snow':[]}

: 

: 

In [ ]:
# for i in range(0,features.shape[0]):
#     # defining a params dict for the parameters to be sent to the API
#     PARAMS = {
#         'lat':features.lat.iloc[i], 
#         'lon':features.lng.iloc[i],
#         'start_date':str(features.trip_Start_time.iloc[i].year)+'-'+str(features.trip_Start_time.iloc[i].month)+'-'+str(features.trip_Start_time.iloc[i].day)+':'+str(features.trip_Start_time.iloc[i].hour),
#         'end_date':str(features.trip_Start_time.iloc[i].year)+'-'+str(features.trip_Start_time.iloc[i].month)+'-'+str(features.trip_Start_time.iloc[i].day)+':'+str(features.trip_Start_time.iloc[i].hour+24),
#         'key':'833701327e23466183824f3bbb10bcad'
#         }
#     # print(str(features.trip_Start_time.iloc[i].year)+'-'+str(features.trip_Start_time.iloc[i].month)+'-'+str(features.trip_Start_time.iloc[i].day)+':'+str(features.trip_Start_time.iloc[i].hour),) 
#     # sending get request and saving the response as response object
#     r = requests.get(url = URL, params = PARAMS)
    
#     # extracting data in json format
#     data = r.json()
  
#     # extracting latitude, longitude and formatted address 
#     # of the first matching location
#     weather['id'] = features['id'].iloc[i]
#     weather['clouds'].append(data['data'][0]['clouds'])
#     weather['dewpt'].append(data['data'][0]['dewpt'])
#     weather['wind_spd'].append(data['data'][0]['wind_spd'])
#     weather['temp'].append(data['data'][0]['temp'])
#     weather['precip'].append(data['data'][0]['precip'])
#     weather['snow'].append(data['data'][0]['snow'])
    

ConnectionError: HTTPSConnectionPool(host='api.weatherbit.io', port=443): Max retries exceeded with url: /v2.0/history/daily?lat=6.6022066&lon=3.2704649&start_date=2021-7-1%3A9&end_date=2021-7-1%3A33&key=833701327e23466183824f3bbb10bcad (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x2cc47cd30>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

: 

: 

In [ ]:
# weather_df = pd.DataFrame(weather)
# print(weather_df.shape)
# weather_df.head()

(0, 6)


,clouds,dewpt,wind_spd,temp,precip,snow


: 

: 

In [15]:
features.to_csv('../data/features.csv',index=False)